In [28]:
import pandas as pd
import numpy as np
from numpy import unique
import os 

import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from numpy import newaxis
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Input
from keras.models import Model
from keras.layers import Input
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt
from keras import layers
import pydot
import pydotplus
from pydotplus import graphviz
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot

In [29]:
train=pd.read_excel('train_data.xlsx')
test=pd.read_excel('test_data.xlsx')

In [30]:
def create_dataset(dataset,look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :-1]
        dataX.append(a)
        dataY.append(dataset[i + look_back, -1])
    return np.asarray(dataX,dtype='float32'), np.asarray(dataY,dtype='float32')

In [31]:
windows=70
train=np.array(train)
test=np.array(test)
train_x,train_y = create_dataset(train,windows)
test_x,test_y = create_dataset(test,windows)
train_y=train_y.reshape(-1,1)
test_y=test_y.reshape(-1,1)

In [32]:
print('shape of training dataset with',windows,'windows...',train_x.shape)
print('shape of test dataset with',windows,'windows...',test_x.shape)
print('shape of training output dataset with',windows,'windows...',train_y.shape)
print('shape of testing output dataset with',windows,'windows...',test_y.shape)

shape of training dataset with 70 windows... (3049, 70, 13)
shape of test dataset with 70 windows... (689, 70, 13)
shape of training output dataset with 70 windows... (3049, 1)
shape of testing output dataset with 70 windows... (689, 1)


In [34]:
verbose, epochs, batch_size = 1, 500, 25
n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
# create a channel for each variable
in_layers, out_layers = list(), list()
for i in range(n_features):
    inputs = Input(shape=(n_timesteps,1))
    lstm1 = layers.LSTM(200, activation='relu')(inputs)
    lstm2 = layers.LSTM(200, activation='relu')(inputs)
    Meta_out = layers.Dense(1)(inputs)
    flat = Flatten()(Meta_out)
    # store layers
    in_layers.append(inputs)
    out_layers.append(flat)
# merge heads
merged = layers.concatenate(out_layers)
print(merged)
# interpretation
dense1 = layers.Dense(200, activation='relu')(merged)
dense2 = layers.Dense(100, activation='relu')(dense1)
outputs = layers.Dense(n_outputs)(dense2)
model = Model(inputs=in_layers, outputs=outputs)
# compile model
model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])
plot_model(model, show_shapes=True, to_file='multiheaded_cnn.png')
# fit network
input_data = [train_x[:,:,i].reshape((train_x.shape[0],n_timesteps,1)) for i in range(n_features)]
model.fit(input_data, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)

Tensor("concatenate_5/concat:0", shape=(None, None), dtype=float32)
Epoch 1/500
3049/3049 [==============================] - 22s 7ms/step - loss: 22.1174 - accuracy: 0.0000e+00
Epoch 2/500
3049/3049 [==============================] - 1s 482us/step - loss: 7.4303 - accuracy: 0.0000e+00
Epoch 3/500
3049/3049 [==============================] - 1s 450us/step - loss: 6.4262 - accuracy: 0.0000e+00
Epoch 4/500
3049/3049 [==============================] - 1s 445us/step - loss: 6.6775 - accuracy: 0.0000e+00
Epoch 5/500
3049/3049 [==============================] - 2s 534us/step - loss: 4.7725 - accuracy: 0.0000e+00
Epoch 6/500
3049/3049 [==============================] - 2s 548us/step - loss: 4.8637 - accuracy: 0.0000e+00
Epoch 7/500
3049/3049 [==============================] - 2s 583us/step - loss: 3.8722 - accuracy: 0.0000e+00
Epoch 8/500
3049/3049 [==============================] - 2s 514us/step - loss: 3.6041 - accuracy: 0.0000e+00
Epoch 9/500
3049/3049 [==============================] - 2s 

3049/3049 [==============================] - 1s 271us/step - loss: 0.0954 - accuracy: 0.0000e+00
Epoch 148/500
3049/3049 [==============================] - 1s 272us/step - loss: 0.0626 - accuracy: 0.0000e+00
Epoch 149/500
3049/3049 [==============================] - 1s 269us/step - loss: 0.0588 - accuracy: 0.0000e+00
Epoch 150/500
3049/3049 [==============================] - 1s 269us/step - loss: 0.0405 - accuracy: 0.0000e+00
Epoch 151/500
3049/3049 [==============================] - 1s 266us/step - loss: 0.0448 - accuracy: 0.0000e+00
Epoch 152/500
3049/3049 [==============================] - 1s 244us/step - loss: 0.0666 - accuracy: 0.0000e+00
Epoch 153/500
3049/3049 [==============================] - 1s 268us/step - loss: 0.0933 - accuracy: 0.0000e+00
Epoch 154/500
3049/3049 [==============================] - 1s 269us/step - loss: 0.0694 - accuracy: 0.0000e+00
Epoch 155/500
3049/3049 [==============================] - 1s 270us/step - loss: 0.0629 - accuracy: 0.0000e+00
Epoch 156/500
3

3049/3049 [==============================] - 1s 282us/step - loss: 0.0231 - accuracy: 0.0000e+00
Epoch 294/500
3049/3049 [==============================] - 1s 277us/step - loss: 0.0514 - accuracy: 0.0000e+00
Epoch 295/500
3049/3049 [==============================] - 1s 276us/step - loss: 0.0463 - accuracy: 0.0000e+00
Epoch 296/500
3049/3049 [==============================] - 1s 274us/step - loss: 0.0577 - accuracy: 0.0000e+00
Epoch 297/500
3049/3049 [==============================] - 1s 273us/step - loss: 0.0521 - accuracy: 0.0000e+00
Epoch 298/500
3049/3049 [==============================] - 1s 291us/step - loss: 0.0342 - accuracy: 0.0000e+00
Epoch 299/500
3049/3049 [==============================] - 1s 277us/step - loss: 0.0304 - accuracy: 0.0000e+00
Epoch 300/500
3049/3049 [==============================] - 1s 274us/step - loss: 0.0213 - accuracy: 0.0000e+00
Epoch 301/500
3049/3049 [==============================] - 1s 273us/step - loss: 0.0250 - accuracy: 0.0000e+00
Epoch 302/500
3

3049/3049 [==============================] - 1s 293us/step - loss: 0.0144 - accuracy: 0.0000e+00
Epoch 440/500
3049/3049 [==============================] - 1s 318us/step - loss: 0.0118 - accuracy: 0.0000e+00
Epoch 441/500
3049/3049 [==============================] - 1s 281us/step - loss: 0.0128 - accuracy: 0.0000e+00
Epoch 442/500
3049/3049 [==============================] - 1s 281us/step - loss: 0.0232 - accuracy: 0.0000e+00
Epoch 443/500
3049/3049 [==============================] - 1s 281us/step - loss: 0.0309 - accuracy: 0.0000e+00
Epoch 444/500
3049/3049 [==============================] - 1s 292us/step - loss: 0.0375 - accuracy: 0.0000e+00
Epoch 445/500
3049/3049 [==============================] - 1s 298us/step - loss: 0.0286 - accuracy: 0.0000e+00
Epoch 446/500
3049/3049 [==============================] - 1s 315us/step - loss: 0.0258 - accuracy: 0.0000e+00
Epoch 447/500
3049/3049 [==============================] - 1s 306us/step - loss: 0.0192 - accuracy: 0.0000e+00
Epoch 448/500
3

In [35]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_53 (InputLayer)           (None, 70, 1)        0                                            
__________________________________________________________________________________________________
input_54 (InputLayer)           (None, 70, 1)        0                                            
__________________________________________________________________________________________________
input_55 (InputLayer)           (None, 70, 1)        0                                            
__________________________________________________________________________________________________
input_56 (InputLayer)           (None, 70, 1)        0                                            
____________________________________________________________________________________________

dense_79 (Dense)                (None, 100)          20100       dense_78[0][0]                   
__________________________________________________________________________________________________
dense_80 (Dense)                (None, 1)            101         dense_79[0][0]                   
Total params: 202,427
Trainable params: 202,427
Non-trainable params: 0
__________________________________________________________________________________________________
